In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
dataset = pd.read_csv('train.csv')

In [3]:
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
dataset.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
dataset=pd.DataFrame(dataset,columns=['Survived','Sex','Age','SibSp','Parch','Fare','Age','Embarked'])

In [6]:
dataset['Age'].fillna(dataset['Age'].mean(), inplace = True)
dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [7]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [8]:
dataset.head()

,Survived,Age,SibSp,Parch,Fare,Age,Sex_male,Embarked_Q,Embarked_S
0,0,22.0,1,0,7.2500,22.0,1,0,1
1,1,38.0,1,0,71.2833,38.0,0,0,0
2,1,26.0,0,0,7.9250,26.0,0,0,1
3,1,35.0,1,0,53.1000,35.0,0,0,1
4,0,35.0,0,0,8.0500,35.0,1,0,1


In [9]:
X = dataset.iloc[:,[1,2,3,4,5,6,7,8]].values
y = dataset.iloc[:, 0].values

In [10]:
from xgboost import XGBClassifier
classifier= XGBClassifier(n_estimators=500, max_depth=3,random_state=101)
classifier.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=101,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [11]:
y_pred=classifier.predict(X)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, y_pred)

In [12]:
cm

array([[526,  23],
       [ 43, 299]])

In [13]:
df1=pd.read_csv('test.csv')

In [14]:
df1.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [15]:
df=pd.DataFrame(df1,columns=['Sex','Age','SibSp','Parch','Fare','Age','Embarked'])

In [16]:
df=pd.get_dummies(df,drop_first=True)

In [17]:
df['Age'].fillna(df['Age'].mean(), inplace = True)

In [18]:
df.head()

,Age,SibSp,Parch,Fare,Age,Sex_male,Embarked_Q,Embarked_S
0,34.5,0,0,7.8292,34.5,1,1,0
1,47.0,1,0,7.0000,47.0,0,0,1
2,62.0,0,0,9.6875,62.0,1,1,0
3,27.0,0,0,8.6625,27.0,1,0,1
4,22.0,1,1,12.2875,22.0,0,0,1


In [19]:
X = df.iloc[:,[0,1,2,3,4,5,6,7]].values

In [20]:
df4=pd.DataFrame()
df4['PassengerId']=df1['PassengerId']

df4['Survived']=classifier.predict(X)
df4.to_csv('sub.csv',index=False)